In [1]:
import pynq
from pynq import Overlay
from pynq import MMIO
import time
from operator import*
import matplotlib.pyplot as plt

# Load the SPI overlay
ol = Overlay("axispi.bit")

In [2]:
print(ol.ip_dict)  # To view the IP block details
 # To view available attributes for the IP block

{'spi_axi_lite_0/S_AXI': {'fullpath': 'spi_axi_lite_0/S_AXI', 'type': 'xilinx.com:module_ref:spi_axi_lite:1.0', 'state': None, 'addr_range': 1073741824, 'phys_addr': 1073741824, 'mem_id': 'S_AXI', 'gpio': {}, 'interrupts': {}, 'parameters': {'BITS_PER_TRANSACTION': '16', 'CLOCKS_PER_BIT': '20', 'Component_Name': 'axispi_spi_axi_lite_0_0', 'EDK_IPTYPE': 'PERIPHERAL', 'C_BASEADDR': '0x40000000', 'C_HIGHADDR': '0x7FFFFFFF'}, 'registers': {}, 'device': <pynq.pl_server.device.XlnkDevice object at 0xb0368710>, 'driver': <class 'pynq.overlay.DefaultIP'>}}


In [3]:
print(dir(spi_axi_ip)) 

NameError: name 'spi_axi_ip' is not defined

In [4]:
# Create an MMIO object manually using the base address and address range
spi_axi_base_address = 0x40000000
spi_axi_addr_range = 0x7FFFFFFF - 0x40000000 + 1  # Calculate address range
spi_axi_mmio = MMIO(spi_axi_base_address, spi_axi_addr_range)

In [5]:
# AXI4-Lite Register Access
class SPI_AXI_Lite:
    def __init__(self, mmio):
        """ Initialize with the MMIO object of the SPI IP block. """
        self.mmio = mmio
        # Assuming base address of the control and status registers
        self.ctrl_reg_offset = 0x00  # Control register offset
        self.status_reg_offset = 0x04  # Status register offset

    def write_ctrl_register(self, value):
        """ Write to the control register using MMIO. """
        self.mmio.write(self.ctrl_reg_offset, value)

    def read_status_register(self):
        """ Read from the status register using MMIO. """
        return self.mmio.read(self.status_reg_offset)

    def spi_transaction(self, data_out):
        """
        Perform an SPI transaction by writing data to the control register
        and reading the response from the status register.
        """
        # Write the data to be transmitted via SPI (Control register)
        self.write_ctrl_register(data_out)

        # Small delay to allow SPI transaction to complete
        time.sleep(0.01)

        # Read the data received from SPI (Status register)
        data_in = self.read_status_register()

        return data_in


In [ ]:
# Instantiate the SPI class with the MMIO object
spi = SPI_AXI_Lite(spi_axi_mmio)

# Perform an SPI transaction
data_out = 0x1234  # Example data to send via SPI
print(f"Sending SPI data: 0x{data_out:X}")
data_in = spi.spi_transaction(data_out)
print(f"Received SPI data: 0x{data_in:X}")